# `logistic-regression-pytorch`

Task: Fit a logistic regression model using PyTorch and fastai's Learner class.

## Setup

In [ ]:
!pip install -Uq fastbook

In [1]:
from fastai.vision.all import *

In [2]:
penguins_csv = download_data('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')
penguins = pd.read_csv(penguins_csv)
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [3]:
len(penguins)

344

In [4]:
penguins = penguins.dropna()

In [5]:
len(penguins)

333

In [6]:
splitter = RandomSplitter(valid_pct=0.2, seed = 2021032)

In [8]:
X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].values
y = penguins['species']

In [10]:
CategoryBlock??

Signature: CategoryBlock(vocab=None, sort=True, add_na=False)
Source:   
def CategoryBlock(vocab=None, sort=True, add_na=False):
    "`TransformBlock` for single-label categorical targets"
    return TransformBlock(type_tfms=Categorize(vocab=vocab, sort=sort, add_na=add_na))
File:      /usr/local/Caskroom/miniconda/base/envs/fastai/lib/python3.8/site-packages/fastai/data/block.py
Type:      function


In [11]:
TransformBlock??

Init signature:
TransformBlock(
    type_tfms=None,
    item_tfms=None,
    batch_tfms=None,
    dl_type=None,
    dls_kwargs=None,
)
Source:        
class TransformBlock():
    "A basic wrapper that links defaults transforms for the data block API"
    def __init__(self, type_tfms=None, item_tfms=None, batch_tfms=None, dl_type=None, dls_kwargs=None):
        self.type_tfms  =            L(type_tfms)
        self.item_tfms  = ToTensor + L(item_tfms)
        self.batch_tfms =            L(batch_tfms)
        self.dl_type,self.dls_kwargs = dl_type,({} if dls_kwargs is None else dls_kwargs)
File:           /usr/local/Caskroom/miniconda/base/envs/fastai/lib/python3.8/site-packages/fastai/data/block.py
Type:           type
Subclasses:     


In [9]:
DataBlock(blocks=[TransformBlock, CategoryBlock])

Init signature:
DataBlock(
    blocks=None,
    dl_type=None,
    getters=None,
    n_inp=None,
    item_tfms=None,
    batch_tfms=None,
    *,
    get_items=None,
    splitter=None,
    get_y=None,
    get_x=None,
)
Source:        
class DataBlock():
    "Generic container to quickly build `Datasets` and `DataLoaders`"
    get_x=get_items=splitter=get_y = None
    blocks,dl_type = (TransformBlock,TransformBlock),TfmdDL
    _methods = 'get_items splitter get_y get_x'.split()
    _msg = "If you wanted to compose several transforms in your getter don't forget to wrap them in a `Pipeline`."
    def __init__(self, blocks=None, dl_type=None, getters=None, n_inp=None, item_tfms=None, batch_tfms=None, **kwargs):
        blocks = L(self.blocks if blocks is None else blocks)
        blocks = L(b() if callable(b) else b for b in blocks)
        self.type_tfms = blocks.attrgot('type_tfms', L())
        self.default_item_tfms  = _merge_tfms(*blocks.attrgot('item_tfms',  L()))
        self.default_

In [ ]:
dls = TabularDataLoaders.from_df(
    penguins,
    procs=[Categorify, Normalize],
    y_names="species",
#    cat_names=['island'],
    cont_names=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
    splitter=splitter,
    bs=10
)
dls.show_batch()

In [ ]:
_, X_batch, y_batch = dls.one_batch()
y_batch = y_batch[:,0].long() # hack to get it in the right shape
X_batch

## Task

Fit a Logistic Regression model on the Palmer Penguins dataset.

## Solution


**Fill in the blanks to construct a `model`**:

```
model = nn.Linear(in_features=..., out_features=..., bias=...)
```

* For `in_features`, think about the shape of the data coming in.
* For `out_features`, think about how many logits we're going to need.

In [ ]:
# your code here
model = nn.Linear(in_features=4, out_features=3, bias=True)

To check that we got it right, **call the `model` with the input data from one example batch**.

Let's get an example batch from the training set dataloader:

Now, give that batch to the model:

In [ ]:
# your code here
model(X_batch)

In [ ]:
model.weight.shape

In [ ]:
model.bias.shape

**Create a `loss_func` by instantiating an `nn.CrossEntropyLoss`.**

In [ ]:
# your code here
loss_func = nn.CrossEntropyLoss()

**Evaluate the loss on the  `loss_func` on the example batch.** 

Note: PyTorch loss functions take inputs, then targets. `sklearn` loss functions (metrics) use the reverse order.

In [ ]:
loss_func(model(X_batch), y_batch)

**Construct a `Learner`.**

* Use the `dataloaders`, `model`, and `loss_func` constructed above.
* Omit `opt_func` to use the default.
* Use `accuracy` as a metric.

In [ ]:
class WrapTabular(nn.Module):
    def __init__(self, module): 
        super().__init__()
        self.module = module
    def forward(self, cats, conts): return self.module(conts)

In [ ]:
learner = TabularLearner(
    dls=dls,
    model=WrapTabular(model),
    loss_func=loss_func,
    metrics=[accuracy],
)

**Train the model for 50 epochs**.

Use a learning rate of 1e-1.

Note: If you want to try alternative learning rates or other parameters here,
call `learner.reset_parameters()` before `fit`.

In [ ]:
learner.fit(50, lr=1e-1)

**Plot the training and validation loss.**

You can use the convenient `plot_loss` method of `learner.recorder`.

In [ ]:
learner.recorder.plot_loss()

**Make a prediction by hand**

In [ ]:
x = X_train[0:1]
model(x)

In [ ]:
y_train[0]

**Was this prediction correct?**

**Use `model.weight` and `model.bias` to make the same prediction**.

In [ ]:
x @ model.weight.T + model.bias

**Compute the predicted probabilities.**

The model outputs logits, so you'll need to use `softmax`. Be careful with dimensionality.

In [ ]:
model(x).softmax(dim=1)

In [ ]:
import sklearn.linear_model

In [ ]:
#clf = sklearn.linear_model.SGDClassifier(
#    loss='log', alpha=0.001, max_iter=100,
#    learning_rate='invscaling', eta0=1e2,
#    verbose=True
#)
clf = sklearn.linear_model.LogisticRegression(C=1.).fit(X[train_indices], y[train_indices])
clf.score(X[valid_indices], y[valid_indices])

In [ ]:
clf.intercept_

In [ ]:
clf.coef_

In [ ]:
model.weight.data.copy_(tensor(clf.coef_))

In [ ]:
model.bias.data.copy_(tensor(clf.intercept_))

In [ ]:
clf.t_

In [ ]:
clf.n_iter_

In [ ]:
alpha = 0.001
typw = np.sqrt(1.0 / np.sqrt(alpha))
# computing eta0, the initial learning rate
initial_eta0 = typw / max(1.0, clf.loss_function_.dloss(-typw, 1.0))
# initialize t such that eta at first sample equals eta0
optimal_init = 1.0 / (initial_eta0 * alpha)

t = np.arange(5400)
eta = 1.0 / (alpha * (optimal_init + t - 1))
plt.plot(t / len(train_indices), eta)

In [ ]:
sklearn.metrics.log_loss(y[valid_indices], clf.predict_proba(X[valid_indices]))

In [ ]:
sklearn.metrics.log_loss

## Analysis